Dataset used:

[Munich Germany](https://mapzen.com/data/metro-extracts/metro/munich_germany/) from OpenStreetMap.  
[Download(OSMXML 39MB)](https://s3.amazonaws.com/metro-extracts.mapzen.com/munich_germany.osm.bz2)

将下载文件解压后命名为"munich_germany.osm"。解压后文件大小为481MB，为了简便测试，使用以下代码来生成小样本：

In [1]:
import sampling
# 生成10分之一的测试数据。
K10 = sampling.writeSample("munich_germany", 10)

运行代码，生成数据大小大约为50MB。

In [3]:
import data

In [4]:
data.process_map(K10, validate=True)